In [ ]:
from pathlib import Path
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from mmwave.util import load_frame
from mmwave.rma import echo_plot
# %matplotlib ipympl

In [ ]:
output_dir = Path("/home/tomin/python/mmwave_postproc/cast_data/small_corner_241121")

frame_file, cfg = load_frame(output_dir)
print(frame_file.shape)

pic_positions = np.mgrid[0 : 0.3 : 1j * 601, 0 : 0.4 : 1j * 801].transpose(1, 2, 0)

dx = cfg.bracket.profile.dx
dy = cfg.bracket.profile.dy
h, w = pic_positions.shape[:2]

In [ ]:
xw = [
    [0, 0, 0, 15, 80, 80, 90, 80, 25, 30, 25, 25, 60, 50, 45, 35],  # 0
    [25, 30, 25, 45, 105, 110, 120, 110, 60, 70, 60, 60, 105, 95, 90, 80],  # 1
    [25, 25, 25, 45, 105, 110, 120, 110, 70, 80, 70, 70, 115, 110, 100, 100],  # 2
    [-30, -25, -30, -10, 55, 65, 70, 65, 20, 35, 25, 30, 75, 65, 65, 55],  # 3
    [-20, -20, -20, 0, 70, 80, 80, 80, 45, 60, 50, 60, 100, 95, 95, 85],  # 4
    [-20, -20, -20, 0, 75, 80, 85, 85, 55, 70, 60, 65, 105, 105, 100, 95],  # 5
    [-120, -120, -115, -95, -20, -10, 0, 0, -30, -15, -25, -28, 22, 18, 20, 5],  # 6
    [-130, -130, -125, -105, -20, -10, -5, 0, -25, -12, -20, -20, 28, 30, 25, 17],  # 7
    [-160, -155, -150, -130, -35, -25, -21, -12, -25, -12, -22, -18, 25, 28, 23, 13],  # 8
    [-90, -90, -90, -75, -15, 0, 0, 0, -45, -25, -40, -35, 5, 0, 0, -15],  # 9
    [-75, -75, -75, -55, 0, 10, 15, 15, -30, -15, -25, -15, 25, 20, 20, 10],  # 10
    [-75, -75, -75, -60, 0, 5, 10, 10, -35, -20, -25, -20, 25, 20, 20, 15],  # 11
]

xw = np.asarray(xw) + 5
xw = np.exp(1j * np.pi * xw / 180)

In [ ]:
tx_idx = widgets.IntSlider(min=0, max=11, step=1, value=0)
rx_idx = widgets.IntSlider(min=0, max=15, step=1, value=0)


@widgets.interact(tx_idx=tx_idx, rx_idx=rx_idx)
def f(tx_idx, rx_idx):
    ID_select = 21
    down_sample = 1
    Echo = (frame_file[rx_idx, tx_idx, :, :, :, 0] + 1j * frame_file[rx_idx, tx_idx, :, :, :, 1])[::down_sample,]
    Sr: np.ndarray = np.fft.fft(Echo)
    Sr[Sr == 0] = 1e-10
    Echo2 = Sr[:, :, ID_select - 1] * xw[tx_idx, rx_idx]
    echo_plot(Echo2, "10 down sample origange", dx, dy)


@widgets.interact(tx_idx=tx_idx, rx_idx=rx_idx)
def f2(tx_idx, rx_idx):
    ID_select = 21
    down_sample = 1
    Echo = (frame_file[rx_idx, tx_idx, :, :, :, 0] + 1j * frame_file[rx_idx, tx_idx, :, :, :, 1])[::down_sample,]
    Sr: np.ndarray = np.fft.fft(Echo)
    Sr[Sr == 0] = 1e-10
    Echo2 = Sr[:, :, ID_select - 1] * xw[tx_idx, rx_idx]
    mmw_line = Echo2[:, 245]
    plt.figure(figsize=[15, 6])
    # plt.plot(np.unwrap(np.angle(mmw_line, deg=True), period=360))
    plt.plot(np.arcsin(np.imag(mmw_line / np.abs(mmw_line))) / np.pi * 180)
    plt.grid()
    # plt.ylim(-180, 180)